# feature はできたのでTargetを作成する
基準日付の終値から20営業日の間の最高値への変化率
https://japanexchangegroup.github.io/J-Quants-Tutorial/

In [1]:
import datetime
import numpy as np
import pandas as pd
import yfinance as yf
import seaborn as sns
import matplotlib.pyplot as plt
import time,sys,os
today = datetime.date.today()

prod = '1y'
dys = 365
dates = '2021-01-08'

last = today +datetime.timedelta(days=-dys)
last = pd.to_datetime(last)

os.makedirs('./csv/'+str(today),exist_ok = True)


In [51]:
"""TOPIX リストの読み込み"""
T100 = pd.read_csv('./csv/JPX_data/TOPIX_100_20210618.csv')
T400 = pd.read_csv('./csv/JPX_data/TOPIX_400_20210618.csv')
S500 = pd.read_csv('./csv/JPX_data/TOPIX_S500_20210618.csv')
Core30 = pd.read_csv('./csv/JPX_data/TOPIX_Core30_20210608.csv')
data =pd.DataFrame(S500) 

stocks = [str(s)+".T" for s in data.code]
#stocks.append("^N225")
tickers = yf.Tickers(" ".join(stocks))
stocks

['1377.T',
 '1379.T',
 '1417.T',
 '1662.T',
 '1719.T',
 '1720.T',
 '1766.T',
 '1786.T',
 '1813.T',
 '1815.T',
 '1821.T',
 '1822.T',
 '1833.T',
 '1835.T',
 '1852.T',
 '1861.T',
 '1881.T',
 '1885.T',
 '1887.T',
 '1890.T',
 '1926.T',
 '1941.T',
 '1942.T',
 '1950.T',
 '1954.T',
 '1961.T',
 '1968.T',
 '1969.T',
 '1973.T',
 '1979.T',
 '2001.T',
 '2109.T',
 '2120.T',
 '2121.T',
 '2146.T',
 '2154.T',
 '2157.T',
 '2168.T',
 '2193.T',
 '2220.T',
 '2222.T',
 '2281.T',
 '2292.T',
 '2296.T',
 '2317.T',
 '2326.T',
 '2327.T',
 '2337.T',
 '2353.T',
 '2379.T',
 '2384.T',
 '2398.T',
 '2427.T',
 '2440.T',
 '2461.T',
 '2471.T',
 '2475.T',
 '2491.T',
 '2590.T',
 '2602.T',
 '2659.T',
 '2664.T',
 '2678.T',
 '2681.T',
 '2685.T',
 '2695.T',
 '2726.T',
 '2730.T',
 '2733.T',
 '2874.T',
 '2910.T',
 '2930.T',
 '2931.T',
 '3002.T',
 '3028.T',
 '3034.T',
 '3036.T',
 '3046.T',
 '3048.T',
 '3050.T',
 '3076.T',
 '3085.T',
 '3087.T',
 '3091.T',
 '3097.T',
 '3098.T',
 '3103.T',
 '3106.T',
 '3107.T',
 '3110.T',
 '3132.T',

In [52]:
len(stocks)

493

In [53]:
closes= []

for i in range(len(tickers.tickers)):
    closes.append(tickers.tickers[i].history(period = prod).Close)

closes = pd.DataFrame(closes).T.ffill()
closes.columns = stocks
print(closes)

                 1377.T       1379.T       1417.T       1662.T      1719.T  \
Date                                                                         
2020-06-18  3378.672607  1963.759766  1565.971069  1859.350464  635.215027   
2020-06-19  3413.351074  1959.869263  1544.465820  1835.013428  630.395508   
2020-06-22  3403.442871  1974.458862  1565.971069  1853.509644  625.575989   
2020-06-23  3413.351074  1962.787109  1564.015991  1855.456665  625.575989   
2020-06-24  3378.672607  1973.486206  1551.308472  1807.755981  610.153442   
...                 ...          ...          ...          ...         ...   
2021-06-14  3855.000000  1937.000000  1937.000000  2008.000000  806.000000   
2021-06-15  3850.000000  1936.000000  1943.000000  1997.000000  814.000000   
2021-06-16  3795.000000  1946.000000  1948.000000  2035.000000  824.000000   
2021-06-17  3765.000000  1926.000000  1929.000000  2034.000000  813.000000   
2021-06-18  3775.000000  1925.000000  1910.000000  1976.000000  

## 予測用データとモデル作成用データに分ける

In [54]:
ttl_closes = closes.copy()
_closes = closes.copy()
feat_closes = _closes.iloc[:-120,:]
val_closes = _closes.iloc[-120:-100,:]
target_closes = _closes.iloc[-100:-20,:]
test_closes = _closes.iloc[-20:,:]


In [55]:
print(feat_closes.shape,val_closes.shape,target_closes.shape,test_closes.shape)

(125, 493) (20, 493) (80, 493) (20, 493)


## 特徴量作成　終値

In [56]:
#終値の営業日リターン
return_1month = feat_closes.copy().pct_change(20).T
return_2month = feat_closes.copy().pct_change(40).T
return_3month = feat_closes.copy().pct_change(60).T
# 終値の営業日ボラティリティ
volatility_1month = feat_closes.copy().diff().rolling(20).std().T
volatility_2month = feat_closes.copy().diff().rolling(40).std().T
volatility_3month = feat_closes.copy().diff().rolling(60).std().T
# 終値と営業日の単純移動平均線の乖離
ma_gap_1month = feat_closes.copy().rolling(20).mean().T
ma_gap_2month = feat_closes.copy().rolling(40).mean().T
ma_gap_3month = feat_closes.copy().rolling(60).mean().T

#終値の営業日リターン
target_return_1month = target_closes.copy().pct_change(20).T
target_return_2month = target_closes.copy().pct_change(40).T
target_return_3month = target_closes.copy().pct_change(60).T
# 終値の営業日ボラティリティ
target_volatility_1month = target_closes.copy().diff().rolling(20).std().T
target_volatility_2month = target_closes.copy().diff().rolling(40).std().T
target_volatility_3month = target_closes.copy().diff().rolling(60).std().T
# 終値と営業日の単純移動平均線の乖離
target_ma_gap_1month = target_closes.copy().rolling(20).mean().T
target_ma_gap_2month = target_closes.copy().rolling(40).mean().T
target_ma_gap_3month = target_closes.copy().rolling(60).mean().T



In [57]:
df_feat = pd.DataFrame()
df_feat['return_1month'] = return_1month.iloc[:,-1]
df_feat['return_2month'] = return_2month.iloc[:,-1]
df_feat['return_3month'] = return_3month.iloc[:,-1]

df_feat['volatility_1month'] = volatility_1month.iloc[:,-1]
df_feat['volatility_2month'] = volatility_2month.iloc[:,-1]
df_feat['volatility_3month'] = volatility_3month.iloc[:,-1]

df_feat['ma_gap_1month'] = ma_gap_1month.iloc[:,-1]
df_feat['ma_gap_2month'] = ma_gap_2month.iloc[:,-1]
df_feat['ma_gap_3month'] = ma_gap_3month.iloc[:,-1]

df_target = pd.DataFrame()
df_target['return_1month'] = target_return_1month.iloc[:,-1]
df_target['return_2month'] = target_return_2month.iloc[:,-1]
df_target['return_3month'] = target_return_3month.iloc[:,-1]

df_target['volatility_1month'] = target_volatility_1month.iloc[:,-1]
df_target['volatility_2month'] = target_volatility_2month.iloc[:,-1]
df_target['volatility_3month'] = target_volatility_3month.iloc[:,-1]

df_target['ma_gap_1month'] = target_ma_gap_1month.iloc[:,-1]
df_target['ma_gap_2month'] = target_ma_gap_2month.iloc[:,-1]
df_target['ma_gap_3month'] = target_ma_gap_3month.iloc[:,-1]

In [58]:
df_feat

,return_1month,return_2month,return_3month,volatility_1month,volatility_2month,volatility_3month,ma_gap_1month,ma_gap_2month,ma_gap_3month
1377.T,0.012863,-0.055828,-0.027374,44.597921,43.223388,59.568319,3444.097327,3533.007312,3612.349768
1379.T,-0.001874,-0.035326,-0.049614,16.010035,20.663877,20.143422,2047.326923,2078.245639,2118.158921
1417.T,0.031174,0.089793,0.059227,23.830445,26.568448,25.013100,1625.193610,1589.130960,1587.300537
1662.T,0.048370,0.072859,0.155203,38.612473,39.701761,40.546545,1811.034979,1774.819217,1752.506793
1719.T,-0.017956,0.034934,-0.032072,10.358845,15.586169,15.060664,699.696027,692.570306,698.533964
...,...,...,...,...,...,...,...,...,...
9945.T,0.023324,0.020349,-0.029314,20.468384,18.627244,20.556864,1695.062811,1696.489883,1715.205811
9946.T,0.006419,0.019509,-0.074147,15.887932,18.635510,18.855310,1377.285522,1376.639999,1400.656551
9948.T,0.008961,-0.047780,-0.161579,30.203096,28.248185,35.147573,2208.841492,2226.305231,2330.065243
9956.T,0.023247,0.002658,-0.092375,42.312759,37.407078,38.922598,2586.778662,2572.357361,2628.526477


In [59]:
df_target

,return_1month,return_2month,return_3month,volatility_1month,volatility_2month,volatility_3month,ma_gap_1month,ma_gap_2month,ma_gap_3month
1377.T,-0.050000,-0.058240,-0.029374,45.598520,54.172289,54.285400,3841.108215,3915.736316,3909.102706
1379.T,-0.029150,-0.080678,-0.065733,19.594508,16.464389,16.106421,2003.000000,2036.276727,2057.299996
1417.T,0.035500,0.011808,0.102627,29.056886,28.431656,26.796214,1794.750000,1813.468573,1785.997890
1662.T,-0.016425,-0.071468,-0.090727,46.396574,44.875343,43.742817,2100.950000,2091.112970,2144.082426
1719.T,-0.003468,-0.027678,0.146451,11.590718,11.763221,12.475550,850.800000,859.659573,849.289245
...,...,...,...,...,...,...,...,...,...
9945.T,0.012220,0.008625,0.054251,36.871898,35.450860,32.795935,1993.400000,1955.100000,1945.843148
9946.T,-0.053170,-0.058943,-0.015038,15.057434,16.739156,19.490254,1425.150000,1443.225000,1440.252413
9948.T,-0.024646,-0.018699,0.018683,38.445416,47.601201,46.361130,2406.900000,2407.300000,2381.927177
9956.T,-0.070204,-0.139335,-0.015278,37.469145,38.627011,39.821735,2342.100000,2419.376984,2430.058447


## 特徴量作成　損益計算書

In [ ]:
'''当期・前期　当期純利益データフレームの作成'''
earnings = []
prev_earnings = []

dummy = tickers.tickers[0].quarterly_financials.T["Net Income"]
dummy[:] = np.nan

for i in range(len(tickers.tickers)):
    try:
        latest_earnings = tickers.tickers[i].quarterly_financials\
        .T["Net Income"]
        previous_earnings = tickers.tickers[i].quarterly_financials\
        .T["Net Income"][1:]
        earnings.append(latest_earnings)
        prev_earnings.append(previous_earnings)
    except:
        earnings.append(dummy)  
        prev_earnings.append(dummy) # エラー発生時はダミーを入れる

earnings = pd.DataFrame(earnings).T  # DataFrame化
prev_earnings = pd.DataFrame(prev_earnings).T  # DataFrame化
earnings.columns = stocks            # カラム名の設定
prev_earnings.columns = stocks            # カラム名の設定

_earnings = earnings.ffill().fillna(0).T.iloc[:,-1]
_prev_earnings = prev_earnings.ffill().fillna(0).T.iloc[:,-1]

print(_earnings)
print(_prev_earnings)

In [21]:
#当期・前 自己資本データフレームの作成
equity   = [] # 自己資本
prev_equity = []

dummy = tickers.tickers[0].quarterly_balance_sheet.T["Total Stockholder Equity"]
dummy[:] = np.nan

for i in range(len(tickers.tickers)):
    try:
        latest_equity = tickers.tickers[i].quarterly_balance_sheet\
        .T["Total Stockholder Equity"]
        previous_equity = tickers.tickers[i].quarterly_balance_sheet\
        .T["Total Stockholder Equity"][1:]
        equity.append(latest_equity)
        prev_equity.append(previous_equity)
    except:
        equity.append(dummy)  
        prev_equity.append(dummy)   # エラー発生時はダミーを入れる

equity = pd.DataFrame(equity).T      # DataFrame化
prev_equity = pd.DataFrame(prev_equity).T   
equity.columns = stocks              # カラム名の設定
prev_equity.columns = stocks 

_equity = equity.ffill().fillna(0).T.iloc[:,-1]
_prev_equity = prev_equity.ffill().fillna(0).T.iloc[:,-1]

print(_equity)
print(_prev_equity)

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [ ]:
#発行株数データフレームの作成
shares   = [] # 発行株数

for i in range(len(tickers.tickers)):
    try:
        shares.append(tickers.tickers[i].info["sharesOutstanding"])
    except:
        shares.append(np.nan)        # エラー発生時はNAN値を入れる

shares = pd.Series(shares)           # Series化
shares.index = stocks                # インデックス名の設定

print(shares)

In [ ]:
#EPS、ROEデータフレームの作成
eps = _earnings/shares.values      # EPS
prev_eps = _prev_earnings/shares.values      # previous EPS
roe = _earnings/_equity    # ROE
prev_roe = _prev_earnings/_prev_equity   # previous ROE

eps = eps.ffill().fillna(0).replace([np.inf,-np.inf],0)                 # 欠損データの補完
prev_eps = prev_eps.ffill().fillna(0) .replace([np.inf,-np.inf],0)  
roe = roe.ffill().fillna(0).replace([np.inf,-np.inf],0)  
prev_roe = prev_roe.ffill().fillna(0).replace([np.inf,-np.inf],0)  

#eps = eps.drop(["^N225"], axis=1) # ^N225カラムは削除しておく
#roe = roe.drop(["^N225"], axis=1)

print(eps)
print(prev_eps)
print(roe)
print(prev_roe)

In [ ]:
df_feat['eps'] = prev_eps
df_feat['roe'] = prev_roe

df_target['eps'] = eps
df_target['roe'] = roe

In [ ]:
df_feat

In [ ]:
df_target

# 目的変数
20日分の株価　四本値 (始値、高値、安値、終値)から
翌営業日以降N（5，10，20）営業日間における最高値及び最安値への変化率
## 120日以前のデータでモデリング
## 119日から100日前の20日分のデータで、120日目の終値と比べてどの程度変化（最高値・最安値）を20，10，5日で算出

In [ ]:
high= []
low = []

for i in range(len(tickers.tickers)):
    high.append(tickers.tickers[i].history(period = prod).High)
    low.append(tickers.tickers[i].history(period = prod).Low)

high = pd.DataFrame(high).T.ffill()
low = pd.DataFrame(low).T.ffill()
high.columns = stocks
low.columns = stocks
print(high)
print(low)

In [ ]:
_high = high.copy()
_low = low.copy()
val_zeroday = _closes.iloc[-121:-120,:] #基準の終値

_val_high = _high.iloc[-120:-100,:]
val_high = pd.concat([val_zeroday,_val_high])

_val_low = _low.iloc[-120:-100,:]
val_low = pd.concat([val_zeroday,_val_low])

test_zeroday = _closes.iloc[-21:-20,:] #基準の終値
_test_high = _high.iloc[-20:,:]
test_high = pd.concat([test_zeroday,_test_high])

_test_low = _low.iloc[-20:,:]
test_low = pd.concat([test_zeroday,_test_low])

In [ ]:
(test_low.iloc[:5,:].min() / test_zeroday).T

In [ ]:
test_zeroday

In [ ]:
df_val = pd.DataFrame()
df_val['zeroday_price'] = val_low.iloc[0,:].T
df_val['label_low_5'] = (val_low.iloc[:5,:].min() / test_zeroday).T
df_val['label_low_10'] = (val_low.iloc[:10,:].min() / test_zeroday).T
df_val['label_low_20'] = (val_low.iloc[:20,:].min() / test_zeroday).T

df_val['label_high_5'] = (val_high.iloc[:5,:].max() / test_zeroday).T
df_val['label_high_10'] = (val_high.iloc[:10,:].max() / test_zeroday).T
df_val['label_high_20'] = (val_high.iloc[:20,:].max() / test_zeroday).T

df_test = pd.DataFrame()
df_test['zeroday_price'] = test_low.iloc[0,:].T
df_test['label_low_5'] = (test_low.iloc[:5,:].min() / test_zeroday).T
df_test['label_low_10'] = (test_low.iloc[:10,:].min() / test_zeroday).T
df_test['label_low_20'] = (test_low.iloc[:20,:].min() / test_zeroday).T

df_test['label_high_5'] = (test_high.iloc[:5,:].max() / test_zeroday).T
df_test['label_high_10'] = (test_high.iloc[:10,:].max() / test_zeroday).T
df_test['label_high_20'] = (test_high.iloc[:20,:].max() / test_zeroday).T


In [ ]:
df_val

In [ ]:
df_test

# Pycaret用データ作成

In [ ]:
df_pycaret = df_feat.copy()
df_pycaret['label_high_20'] = df_val['label_high_20']

In [ ]:
df_pycaret

In [ ]:
from pycaret.regression import*

In [ ]:
mdl1 = setup(df_pycaret, target='label_high_20',ignore_features = None)

In [ ]:
compare_models()

In [ ]:
#llar = create_model('llar')

In [ ]:
#lightgbm = create_model('lightgbm')